In [4]:
import pandas as pd
import cv2 as cv
import numpy as np
import os

In [15]:
df = pd.read_csv('train_boundingboxes.csv',sep=', ')

/tmp/ipykernel_16734/1907980466.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('train_boundingboxes.csv',sep=', ')


In [3]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)
imglist = sorted_alphanumeric(os.listdir('Train'))
widthList = []
heightList = []
for img in imglist:
    image = cv.imread('Train/'+img)
    height, width, channels = image.shape
    widthList.insert(len(widthList),width)
    heightList.insert(len(heightList),height)

In [22]:
def findX(xmin,xmax,width):
    return (xmin + xmax)/ 2.0 / width

def findY(ymin,ymax,height):
    return (ymin + ymax)/ 2.0 / height

def findW(xmin,xmax,width):
    return abs(xmin - xmax) / width

def findH(ymin,ymax,height):
    return abs(ymin - ymax) / height

df['pwidth'] = widthList
df['pheight'] = heightList
df['x_centre'] = df.apply(lambda x: findX(x['X_min'],x['X_max'],x['pwidth']),axis=1)
df['y_centre'] = df.apply(lambda x: findY(x['Y_min'],x['Y_max'],x['pheight']),axis=1)
df['width'] = df.apply(lambda x: findW(x['X_min'],x['X_max'],x['pwidth']),axis=1)
df['height'] = df.apply(lambda x: findH(x['Y_min'],x['Y_max'],x['pheight']),axis=1)
df['Class'] = 0

In [31]:
dfresult = df[['Class','x_centre','y_centre','width','height']]
dfresult = dfresult.round(6)
dfresult = dfresult.astype(str)

In [32]:
i=0
for row in dfresult.values:
    filename = 'data/obj/{}.txt'.format(i)
    row.tofile(filename, sep=" ", format="%s")
    i+=1

In [7]:
path = 'data/obj/'
imgList = sorted_alphanumeric(os.listdir('Train'))
textFile = open('data/train.txt','w')
testFile = open('data/test.txt','w')
total =1000
test = 100
i = 0
for img in imgList:
    if(i < total - test):
        imgPath = path+img+'\n'
        textFile.write(imgPath)
        i+=1
    else:
        imgPath = path+img+'\n'
        testFile.write(imgPath)